# Load the code search model into the Solr search engine

In this hackathon, we will be generating code with AI/ML provided by the well established [computate project](https://github.com/computate-org/computate). We will automatically generate OpenAPI specs, database table schemas, Java POJOs, Vert.x Reactive Java APIs, Handlebars HTML page templates, JavaScript page and API functions, NGSI-LD Context data, OpenShift Custom Resource Definitions, and Ansible Operator roles and playbooks for any of the hundreds of open source Edge device data related [FIWARE Smart Data Models available here](https://github.com/smart-data-models). 

## Prerequisites
To run the AI/ML code generation, you need to have completed the steps in the [README](README.md), [notebook 01-install-prerequisites.ipynb](01-install-prerequisites.ipynb), and [ notebook 02-deploy-microservices.ipynb](01-install-prerequisites.ipynb) to install the dependencies into the workbench, and deploy all the microservices. Then run the commands below in your OpenShift AI Workbench. 

## Clone Computate and Smart Village repos

### Install the computate_project Ansible Role

The [computate_project Ansible Role](https://github.com/computate-org/computate_project.git) is for installing Java projects that want to generate code for them using the [computate project](https://github.com/computate-org/computate.git). 

In [ ]:
install -d ~/.ansible/roles
git clone https://github.com/computate-org/computate_project.git ~/.ansible/roles/computate.computate_project
echo DONE

### Install the computate project

The [computate project](https://github.com/computate-org/computate.git) is a Java project that watches for changes to files in a directory recursively, parses the Java code as it's created or updated, indexes every detail about each Java class, constructor, method, and field in the Apache Solr search engine, and generates code based on what it discovers. This is the main open source library that does the AI/ML code generation. Instead of a typical model server, we use Apache Solr as the model server. We can flexibly index every aspect about our Java code in Solr as we code over time, including multiple Java projects at the same time, and link together classes, types, and foreign key relations between projects. 

In [ ]:
git clone https://github.com/computate-org/computate.git ~/computate
echo DONE

Use maven to compile and install the computate project in the workbench. 

In [ ]:
(cd ~/computate &&  mvn clean install)
echo DONE

### Install the computate-search project

The [computate-search project](https://github.com/computate-org/computate-search.git) is a Java project that mainly interacts with Apache Solr to make search queries, and parse the response. There are also several date time serializers and deserializers for handling date times in requests. One other important class is the `Wrap` class which can wrap a Java field initialization value by it's generic type as part of the generated initialization code for each class. 

In [ ]:
git clone https://github.com/computate-org/computate-search.git ~/computate-search
echo DONE

Run the Ansible Playbook to configure and compile the `computate-search` project. 

In [ ]:
ansible-playbook ~/.ansible/roles/computate.computate_project/install.yaml \
  -e SITE_NAME=computate-search \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/computate-search" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
echo DONE

Take a look at the `computate-search/config/computate-search.yaml` configuration file that was created based on your environment. 

In [ ]:
cat ~/computate-search/config/computate-search.yaml
echo DONE

Run the `index.sh` shell script of the `computate` project on  the  `computate-search` project to index all of the `computate-search` Java classes into the Apache Solr search engine. 

In [ ]:
env SITE_NAME=computate-search \
  SITE_PATH=$HOME/computate-search \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/index.sh
echo DONE

Now query the Solr search engine to find out how many Java classes, constructors, methods, fields, and generated fields are found in the  `computate-search` project. 

In [ ]:
curl -s 'http://solr:8983/solr/computate/query?rows=0&fq=siteNom_indexed_string:computate-search' -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

### Install the computate-vertx project

The [computate-vertx project](https://github.com/computate-org/computate-vertx.git) is a Java project that integrates Vert.x libraries into a project for reactive, event-bus driven software development. There is additional search capabilities built in to perform searches to Apache Solr asynchronously. There is PostgreSQL database integration with serializers and deserializers for GeoJson fields. Additional Vert.x tools, OpenAPI schema writers, boiler plate code project writers, FIWARE and NGSI-LD context writers, OpenID Connect/OAuth2 authentication/authorization handlers, Handlebars HTML Template handlers, and a base API Java Interface for building powerful, reactive secure APIs for anything. 

In [ ]:
git clone https://github.com/computate-org/computate-vertx.git ~/computate-vertx
echo DONE

Run the Ansible Playbook to configure and compile the `computate-vertx` project. 

In [ ]:
ansible-playbook ~/.ansible/roles/computate.computate_project/install.yaml \
  -e SITE_NAME=computate-vertx \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/computate-vertx" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
echo DONE

Take a look at the `computate-vertx/config/computate-search.yaml` configuration file that was created based on your environment. 

In [ ]:
cat ~/computate-vertx/config/computate-vertx.yaml
echo DONE

Run the `index.sh` shell script of the `computate` project on  the  `computate-vertx` project to index all of the `computate-vertx` Java classes into the Apache Solr search engine. 

In [ ]:
env SITE_NAME=computate-vertx \
  SITE_PATH=$HOME/computate-vertx \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/index.sh
echo DONE

Now query the Solr search engine to find out how many Java classes, constructors, methods, fields, and generated fields are found in the  `computate-vertx` project. 

In [ ]:
curl -s 'http://solr:8983/solr/computate/query?rows=0&fq=siteNom_indexed_string:computate-vertx' -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

### Install the smartvillage-platform project

The [smartvillage-platform project](https://github.com/computate-org/smartvillage-platform.git) is a Java project that integrates the computate-vertx and computate-search projects together with additional tools for smart cities. There are helper classes for working with latitude/longitude coordinates, angles and directions of vehicles, map results, models, search engine results, articles, page layouts, site users, and manages the package deployment of new release versions of the Smart Village Platform. 

In [ ]:
git clone https://github.com/computate-org/smartvillage-platform.git ~/smartvillage-platform
git clone https://github.com/computate-org/smartvillage-platform-static.git ~/smartvillage-platform-static
echo DONE

Run the Ansible Playbook to configure and compile the `smartvillage-platform` project. 

In [ ]:
ansible-playbook ~/.ansible/roles/computate.computate_project/install.yaml \
  -e SITE_NAME=smartvillage-platform \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/smartvillage-platform" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
echo DONE

Take a look at the `smartvillage-platform/config/smartvillage-platform.yaml` configuration file that was created based on your environment. 

In [ ]:
cat ~/smartvillage-platform/config/smartvillage-platform.yaml
echo DONE

Run the `index.sh` shell script of the `computate` project on  the  `smartvillage-platform` project to index all of the `smartvillage-platform` Java classes into the Apache Solr search engine. 

In [ ]:
env SITE_NAME=smartvillage-platform \
  SITE_PATH=$HOME/smartvillage-platform \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/index.sh
echo DONE

Now query the Solr search engine to find out how many Java classes, constructors, methods, fields, and generated fields are found in the  `smartvillage-platform` project. 

In [ ]:
curl -s 'http://solr:8983/solr/computate/query?rows=0&fq=siteNom_indexed_string:smartvillage-platform' -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

### Install the smartabyar-smartvillage project

The [smartabyar-smartvillage project](https://github.com/computate-org/smartabyar-smartvillage.git) is a Java project that integrates the computate-vertx, computate-search, and smartvillage-platform projects together with additional tools for the village of Veberöd Sweden. We deploy smart devices to the application based on [FIWARE's Smart Data Models](https://github.com/smart-data-models) like [TrafficFlowObserved](https://github.com/smart-data-models/dataModel.Transportation/tree/f0eb02999cf6a4a29082e8bcd7fecb134264ae58/TrafficFlowObserved), and [CrowdFlowObserved](https://github.com/smart-data-models/dataModel.Transportation/tree/f0eb02999cf6a4a29082e8bcd7fecb134264ae58/CrowdFlowObserved) to track vehicle data moving through lanes of traffic, and count the number of people at a particular crosswalk. We also build new Smart Data Models like [TrafficSimulation](https://github.com/smartabyar-smartvillage/smartabyar-smartvillage-static/tree/main/fiware/TrafficSimulation), and [SmartTrafficLight](https://github.com/smartabyar-smartvillage/smartabyar-smartvillage-static/tree/main/fiware/SmartTrafficLight) to research, simulate, and prototype what real SmartTrafficLight data would look like in the village of Veberöd. We also build Simulation Reports to adjust parameters and report on the performance of traffic lights in an area based on running repeated simulations. 

In [ ]:
git clone https://github.com/computate-org/smartabyar-smartvillage.git ~/smartabyar-smartvillage
git clone https://github.com/computate-org/smartabyar-smartvillage-static.git ~/smartabyar-smartvillage-static
echo DONE

Run the Ansible Playbook to configure and compile the `smartabyar-smartvillage` project. 

In [ ]:
SITE_HOST_NAME="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get route/smartvillage-web -o jsonpath={.spec.host})"
AUTH_CLIENT="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get secret/smartvillage -o jsonpath={.data.CLIENT_ID} | base64 -d)"
AUTH_SECRET="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get secret/smartvillage -o jsonpath={.data.CLIENT_SECRET} | base64 -d)"

ansible-playbook ~/.ansible/roles/computate.computate_project/install.yaml \
  -e SITE_NAME=smartabyar-smartvillage \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME_COMPUTATE=solr \
  -e SOLR_PORT_COMPUTATE=8983 \
  -e SOLR_SSL_COMPUTATE=false \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/smartabyar-smartvillage" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e ZOOKEEPER_HOST_NAME=zookeeper \
  -e ZOOKEEPER_PORT=2181 \
  -e JDBC_HOST=postgres-smartvillage \
  -e JDBC_PORT=5432 \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SITE_BASE_URL="https://${SITE_HOST_NAME}" \
  -e STATIC_BASE_URL="https://${SITE_HOST_NAME}/static" \
  -e AUTH_CLIENT="${AUTH_CLIENT}" \
  -e AUTH_SECRET="${AUTH_SECRET}" \
  -e AUTH_ROLE_ADMIN="Public" \
  -e ENABLE_KAFKA=false \
  -e ENABLE_IMPORT_DATA=false
echo DONE

Take a look at the `smartabyar-smartvillage/config/smartabyar-smartvillage.yaml` configuration file that was created based on your environment. 

In [ ]:
cat ~/smartabyar-smartvillage/config/smartabyar-smartvillage.yaml
echo DONE

Run the `index.sh` shell script of the `computate` project on  the  `smartabyar-smartvillage` project to index all of the `smartabyar-smartvillage` Java classes into the Apache Solr search engine, and regenerate all the generated code for the project. 

In [ ]:
env SITE_NAME=smartabyar-smartvillage \
  SITE_PATH=$HOME/smartabyar-smartvillage \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/index.sh
echo DONE

Now query the Solr search engine to find out how many Java classes, constructors, methods, fields, and generated fields are found in the  `smartabyar-smartvillage` project. 

In [ ]:
curl -s 'http://solr:8983/solr/computate/query?rows=0&fq=siteNom_indexed_string:smartabyar-smartvillage' -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

### Clone and index all Smart Data Models

Run the Ansible Playbook below to clone the hundreds of FIWARE Smart Data Models, and index each Smart Data Model in Apache Solr. We search this Smart Data Model data when we create new Java classes to see if there is a `SmartDataModel: search terms...` class comment in the code that closely matches a Smart Data Model in the Solr search engine, then we will generate all the code for this Smart Data Model automatically. 

In [ ]:
ansible-playbook ~/smartvillage-operator/clone-smart-model-data.yaml -e PROJECT_PREFIX="$HOME" -e SOLR_BASE_URL="http://solr:8983"
echo DONE